# 연관 검색어 크롤링

## 1. 다음(https://www.daum.net/) 사이트에서 '행복 일러스트' 검색시 연관 검색어 출력

In [2]:
import pandas as pd
from selenium import webdriver
import requests # url 로부터 html 문서를 가져옴
from selenium.webdriver.common.keys import Keys 
import bs4
from bs4 import BeautifulSoup # Html 문서에서 데이터를 추출
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
driver= webdriver.Chrome("C:/Users/CHA-LAPTOP/Downloads/chromedriver")
url= "https://www.daum.net/"
driver.get(url)
searchbox= driver.find_element_by_xpath('//*[@id="q"]') #  다음검색창 xpath
word= "행복 일러스트"
searchbox.send_keys(word) # 행복 일러스트 입력
searchbox.send_keys(Keys.ENTER) # 엔터
print(driver.current_url) 
url2= driver.current_url # 현재 url 주소 저장

https://search.daum.net/search?w=tot&DA=YZR&t__nil_searchbox=btn&sug=&sugo=&sq=&o=&q=%ED%96%89%EB%B3%B5+%EC%9D%BC%EB%9F%AC%EC%8A%A4%ED%8A%B8


In [4]:
# 연관 검색어 불러오기
r= requests.get(url2)
bs4_r= bs4.BeautifulSoup(r.text, 'lxml')
div_inner= bs4_r.find('div', class_='list_keyword type2')
try:
    div_inner= bs4_r.find('div',class_='list_keyword type2')
    span_list= div_inner.find_all('span', class_='wsn')
except:
    if pd.isna(div_inner):
        try:
            div_inner= bs4_r.find('div', class_='list_keyword')
            span_list= div_inner.find_all('span',class_='wsn')
        except:
            print(word_list[i], '연관 검색어가 없습니다.')
            span_list=[]
# 연관 검색어 출력
related_word_list= []
for i,word in enumerate(span_list):
    related_word_list.append(span_list[i].text)
related_word_list

['사랑 일러스트',
 '감사 일러스트',
 '웃음 일러스트',
 '연인 일러스트',
 '건강 일러스트',
 '가족 일러스트',
 '스마일 일러스트',
 '하트 일러스트',
 '꽃 일러스트',
 '행복한 가정 일러스트',
 '친구 일러스트',
 '교회 일러스트',
 '마음 일러스트',
 '희망 일러스트',
 '웃는 얼굴 일러스트',
 '행복 이미지',
 '무지개 일러스트',
 '나무 일러스트',
 '여름 일러스트',
 '평화 일러스트']

## 2. crawlig_related_word 함수로 연관 검색어 수집

## 단어 검색 -> 연관 검색어 저장 -> 나온 연관검색어 검색 -> 연관 검색어 저장 (3번 반복)

In [5]:
def crawlig_related_word(word_list, n):
    if n==0:
        print("크롤링 종료")
    else:
        merge_df = pd.DataFrame()
        for i in range(len(word_list)):
           temp_df= pd.DataFrame()
           temp_list= []
            # 앞부분과 동일
           driver= webdriver.Chrome("C:/Users/CHA-LAPTOP/Downloads/chromedriver")
           url2= "https://www.daum.net/"
           driver.get(url2)
           searchbox= driver.find_element_by_xpath('//*[@id="q"]')
           word= word_list[i]
           searchbox.send_keys(word)
           searchbox.send_keys(Keys.ENTER)
           url2= driver.current_url
           r= requests.get(url2)
           bs4_r= bs4.BeautifulSoup(r.text, 'lxml')
           print(word_list[i])
           div_inner= bs4_r.find('div', class_='list_keyword type2')
            # 저장 -> 검색 반복
           try:
               div_inner= bs4_r.find('div',class_='list_keyword type2')
               span_list= div_inner.find_all('span', class_='wsn')
           except:
               if pd.isna(div_inner):
                   try:
                       div_inner= bs4_r.find('div',class_='list_keyword')
                       span_list= div_inner.find_all('span',class_='wsn')
                   except:
                       print(word_list[i],'연관 검색어가 없습니다')
                       span_list= []
           if len(span_list) ==0:
               pass
           else:
               for k,word in enumerate(span_list):
                   temp_list.append(span_list[k].text)
               temp_df['sub_word']= temp_list
               temp_df['sup_word']= word_list[i]
               temp_df= pd.DataFrame(temp_df, columns=['sup_word', 'sub_word'])
               merge_df= pd.concat([merge_df,temp_df],axis=0)
        # 딕셔너리 형태로 저장    
        dictionary[n]= merge_df
        pass_word_list= merge_df['sub_word'].values
        print('!!!!!!!!!!!!!!!크롤링 시작!!!!!!!!!!!!!')
        crawlig_related_word(pass_word_list, n-1)

In [7]:
# 실행시 30분 이상 소요됨 
word= ['행복 일러스트']
dictionary={}
crawlig_related_word(word, 3)

행복 일러스트
!!!!!!!!!!!!!!!크롤링 시작!!!!!!!!!!!!!
사랑 일러스트
감사 일러스트
웃음 일러스트
연인 일러스트
건강 일러스트
가족 일러스트
스마일 일러스트
하트 일러스트
꽃 일러스트
행복한 가정 일러스트
친구 일러스트
교회 일러스트
마음 일러스트
희망 일러스트
웃는 얼굴 일러스트
행복 이미지
무지개 일러스트
나무 일러스트
여름 일러스트
평화 일러스트
!!!!!!!!!!!!!!!크롤링 시작!!!!!!!!!!!!!
하트 일러스트
행복 일러스트
연인 일러스트
사랑나눔 일러스트
나무 일러스트
우정 일러스트
꽃 일러스트
가족 일러스트
어린이 일러스트
웃음 일러스트
친구 일러스트
사람 일러스트
감사 일러스트
편지 일러스트
희망 일러스트
섬김 일러스트
구름 일러스트
결혼 일러스트
아기 일러스트
예수님 일러스트
인사 일러스트
감사합니다 일러스트
추수감사 일러스트
웃음 일러스트
축하 일러스트
사랑 일러스트
안녕 일러스트
감사 이미지
행복 일러스트
예쁜 꽃 일러스트
입학 일러스트
하트 일러스트
추석 일러스트
기도 일러스트
박수 일러스트
카네이션 일러스트
스마일 일러스트
미소 일러스트
웃음 이모티콘
웃는 얼굴 이미지
봄 일러스트
가족 일러스트
웃는 아이 일러스트
행복 일러스트
애인 일러스트
키스 일러스트
웨딩 일러스트
아름다운 연인 일러스트
남녀 일러스트
커플 일러스트
연인 일러스트 이미지
결혼 일러스트
일러스트레이터
일러스트란
가족 일러스트
운동 일러스트
코로나 일러스트
행복 일러스트
건강 이미지
달리기 일러스트
친구 일러스트
음식 일러스트
병원 일러스트
결혼 일러스트
줄넘기 일러스트
학교 일러스트
화이팅 일러스트
행복한 가족 일러스트
우리가족 일러스트
친구 일러스트
집 일러스트
부모님 일러스트
엄마 일러스트
대가족 일러스트
어린이집 일러스트
사랑 일러스트
부부 일러스트
노인 일러스트
과학 일러스트
카네이션 일러스트
아빠 일러스트
가족여행 일러스트
어버이날 일러스트
엄마 아빠 일러스트
가족사진 일러스트
자매 일러스트
가족 운동회 일러스트
웃는 얼굴 일러스

In [9]:
# dictionary -> dataframe으로 변경
a= pd.DataFrame(dictionary[3])
b= pd.DataFrame(dictionary[2])
c= pd.DataFrame(dictionary[1])
a.columns=['1st crawl','2nd crawl']
b.columns=['2nd crawl','3rd crawl']
c.columns=['3rd crawl','4th crawl']
d= pd.merge(a,b)
e= pd.merge(d,c)
e

,1st crawl,2nd crawl,3rd crawl,4th crawl
0,행복 일러스트,사랑 일러스트,하트 일러스트,사랑 일러스트
1,행복 일러스트,사랑 일러스트,하트 일러스트,손하트 일러스트
2,행복 일러스트,사랑 일러스트,하트 일러스트,꽃 일러스트
3,행복 일러스트,사랑 일러스트,하트 일러스트,예쁜 하트 일러스트
4,행복 일러스트,사랑 일러스트,하트 일러스트,풍선 일러스트
...,...,...,...,...
10049,행복 일러스트,평화 일러스트,삽화 일러스트,삽화 그림
10050,행복 일러스트,평화 일러스트,삽화 일러스트,일러스트 기초
10051,행복 일러스트,평화 일러스트,삽화 일러스트,얼굴 표정 일러스트
10052,행복 일러스트,평화 일러스트,삽화 일러스트,멋진 그림 일러스트


## 3. 전처리

In [10]:
# 일러스트, 이미지, 이모티콘, 일러스트레이터 단어 삭제
# 일러스트 제거
e['1st crawl']=e['1st crawl'].apply(lambda e: e.split("일러스트")[0])
e['2nd crawl']=e['2nd crawl'].apply(lambda e: e.split("일러스트")[0])
e['3rd crawl']=e['3rd crawl'].apply(lambda e: e.split("일러스트")[0])
e['4th crawl']=e['4th crawl'].apply(lambda e: e.split("일러스트")[0])
# 이미지 제거
e['1st crawl']=e['1st crawl'].apply(lambda e: e.split("이미지")[0])
e['2nd crawl']=e['2nd crawl'].apply(lambda e: e.split("이미지")[0])
e['3rd crawl']=e['3rd crawl'].apply(lambda e: e.split("이미지")[0])
e['4th crawl']=e['4th crawl'].apply(lambda e: e.split("이미지")[0])
# 이모티콘 제거
e['1st crawl']=e['1st crawl'].apply(lambda e: e.split("이모티콘")[0])
e['2nd crawl']=e['2nd crawl'].apply(lambda e: e.split("이모티콘")[0])
e['3rd crawl']=e['3rd crawl'].apply(lambda e: e.split("이모티콘")[0])
e['4th crawl']=e['4th crawl'].apply(lambda e: e.split("이모티콘")[0])
# 일러스트레이터 제거
e['1st crawl']=e['1st crawl'].apply(lambda e: e.split("일러스트레이터")[0])
e['2nd crawl']=e['2nd crawl'].apply(lambda e: e.split("일러스트레이터")[0])
e['3rd crawl']=e['3rd crawl'].apply(lambda e: e.split("일러스트레이터")[0])
e['4th crawl']=e['4th crawl'].apply(lambda e: e.split("일러스트레이터")[0])

In [11]:
# 전체파일 csv 파일로 저장
e.to_csv('craw_preprocessing.csv')
# 마지막 연관검색어 결과만 따로 저장
e['4th crawl'].to_csv('craw_preprocessing4th.csv')

In [12]:
# 마지막 연관검색어 주요 단어 수 출력
from konlpy.tag import Okt
from collections import Counter
f = open('craw_preprocessing4th.csv', 'r', encoding='utf-8')
news= f.read()
# okt 객체 생성
okt= Okt()
noun= okt.nouns(news)
count= Counter(noun)
# 가장 최빈단어 100개 출력
noun_list= count.most_common(100)
for v in noun_list:
    print(v)

('하트', 836)
('꽃', 512)
('친구', 416)
('나무', 379)
('그림', 374)
('사랑', 339)
('얼굴', 310)
('가족', 295)
('행복', 233)
('감사', 223)
('여름', 215)
('웃음', 194)
('도안', 176)
('희망', 161)
('사람', 159)
('연인', 154)
('어린이', 154)
('예수님', 118)
('스마일', 114)
('배경', 114)
('우정', 111)
('봄', 107)
('풍선', 103)
('구름', 100)
('별', 99)
('아기', 97)
('무지개', 96)
('손', 95)
('결혼', 93)
('카네이션', 92)
('아이', 91)
('메모', 90)
('어린이집', 88)
('건강', 87)
('교회', 87)
('유치원', 84)
('아이콘', 82)
('편지', 82)
('나눔', 82)
('테두리', 81)
('캐릭터', 79)
('자동차', 75)
('겨울', 74)
('해바라기', 74)
('가정', 71)
('동그라미', 68)
('마음', 66)
('평화', 65)
('동물', 61)
('학교', 56)
('꽃다발', 53)
('풍경', 53)
('엄마', 52)
('기도', 50)
('가을', 47)
('새싹', 44)
('축하', 44)
('여름방학', 42)
('여자', 41)
('아빠', 40)
('나뭇잎', 37)
('나비', 37)
('봄꽃', 37)
('화분', 36)
('자연', 36)
('하늘', 36)
('어깨동무', 34)
('인사', 33)
('벚꽃', 32)
('그리기', 32)
('숲', 30)
('사진', 30)
('어버이날', 29)
('꽃잎', 28)
('집', 27)
('입학', 26)
('수채', 25)
('공동체', 25)
('일상', 25)
('가족사진', 24)
('구글', 24)
('아트', 24)
('우리', 23)
('부부', 23)
('열매', 22)
('선생님', 22)
('장미',

## 4. Mlxtend (Machine Learning Extensions) 모듈
### http://rasbt.github.io/mlxtend/



In [35]:
x= pd.read_csv("craw_preprocessing.csv", encoding='utf-8')
x= x[['1st crawl','2nd crawl','3rd crawl','4th crawl']]
x

,1st crawl,2nd crawl,3rd crawl,4th crawl
0,행복,사랑,하트,사랑
1,행복,사랑,하트,손하트
2,행복,사랑,하트,꽃
3,행복,사랑,하트,예쁜 하트
4,행복,사랑,하트,풍선
...,...,...,...,...
10049,행복,평화,삽화,삽화 그림
10050,행복,평화,삽화,NaN
10051,행복,평화,삽화,얼굴 표정
10052,행복,평화,삽화,멋진 그림


In [41]:
# df -> list 형태로 변환
x.dropna(axis=0, inplace=True)
val = x.values
dataset = val.tolist()
dataset[:10]

[['행복 ', '사랑 ', '하트 ', '사랑 '],
 ['행복 ', '사랑 ', '하트 ', '손하트 '],
 ['행복 ', '사랑 ', '하트 ', '꽃 '],
 ['행복 ', '사랑 ', '하트 ', '예쁜 하트 '],
 ['행복 ', '사랑 ', '하트 ', '풍선 '],
 ['행복 ', '사랑 ', '하트 ', '하트 그림'],
 ['행복 ', '사랑 ', '하트 ', '나무 '],
 ['행복 ', '사랑 ', '하트 ', '별 '],
 ['행복 ', '사랑 ', '하트 ', '친구 '],
 ['행복 ', '사랑 ', '하트 ', '귀여운 ']]

In [56]:
# Apriori 알고리즘을 사용하여 지지도 신뢰도 향상도 출력
# pip install mlxtend
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [50]:
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,2020년 새해인사,5월,5월 가정의 달,5월 풍경,6월,7월,7월의,8월 인사,9월,cu 카네이션,...,환영,환자,환한 웃음,환호,활짝 웃는,회사,휴식,희망,힐링,힘내
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9784,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9785,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9786,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9787,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [46]:
# 항목 갯수가 1개 & 지지도(support)가 0.01 이상인 항목 10개 추출
# Support(지지도) : 전체 Data에서 해당 단어를 검색 할 확률
# 지지도 = P(A∩B)
frequent_itemsets= apriori(df, min_support=0.01, use_colnames=True)
frequent_itemsets['length']= frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets[(frequent_itemsets['length']==1) & \
    (frequent_itemsets['support'] >= 0.01)].sort_values(by='support',ascending=False).head(10)

,support,itemsets,length
39,1.000000,(행복 ),1
18,0.284401,(사랑 ),1
36,0.210747,(하트 ),1
33,0.150577,(친구 ),1
8,0.119318,(꽃 ),1
1,0.110737,(감사 ),1
10,0.109817,(나무 ),1
0,0.103484,(가족 ),1
13,0.078558,(무지개 ),1
42,0.069057,(희망 ),1


In [52]:
# 항목 갯수가 2개 & 지지도(support)가 0.01 이상인 항목 10개 추출
frequent_itemsets= apriori(df, min_support=0.01, use_colnames=True)
frequent_itemsets['length']= frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets[(frequent_itemsets['length']==2) & \
    (frequent_itemsets['support'] >= 0.01)].sort_values(by='support',ascending=False).head(10)

,support,itemsets,length
92,0.284401,"(행복 , 사랑 )",2
115,0.210747,"(행복 , 하트 )",2
111,0.150577,"(행복 , 친구 )",2
65,0.119318,"(행복 , 꽃 )",2
49,0.110737,"(행복 , 감사 )",2
69,0.109817,"(행복 , 나무 )",2
46,0.103484,"(행복 , 가족 )",2
76,0.078558,"(무지개 , 행복 )",2
120,0.069057,"(희망 , 행복 )",2
56,0.068853,"(행복 , 교회 )",2


In [53]:
# 항목 갯수가 3개 & 지지도(support)가 0.01 이상인 항목 10개 추출
frequent_itemsets= apriori(df, min_support=0.01, use_colnames=True)
frequent_itemsets['length']= frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets[(frequent_itemsets['length']==3) & \
    (frequent_itemsets['support'] >= 0.01)].sort_values(by='support',ascending=False).head(10)

,support,itemsets,length
151,0.049648,"(행복 , 하트 , 사랑 )",3
150,0.042292,"(행복 , 친구 , 사랑 )",3
121,0.038308,"(행복 , 가족 , 사랑 )",3
124,0.035141,"(행복 , 감사 , 사랑 )",3
155,0.034733,"(행복 , 친구 , 하트 )",3
134,0.032588,"(행복 , 꽃 , 하트 )",3
136,0.031975,"(행복 , 하트 , 나무 )",3
153,0.028910,"(희망 , 행복 , 사랑 )",3
123,0.024517,"(행복 , 친구 , 가족 )",3
131,0.021759,"(행복 , 꽃 , 나무 )",3


In [59]:
# 신뢰도 0.7이상인 항목 15개 추출
# Confidence(신뢰도) : 어떤 단어를 검색했을때 다른 단어도 검색 할 확률
# 신뢰도 = P(A∩B) / P(A)

association_rules(frequent_itemsets, metric='confidence', min_threshold=0.7).sort_values(by='lift', ascending=False).head(15)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
78,(아기 ),"(행복 , 사랑 )",0.010931,0.284401,0.010522,0.962617,3.384718,0.007413,19.142277
76,"(행복 , 아기 )",(사랑 ),0.010931,0.284401,0.010522,0.962617,3.384718,0.007413,19.142277
21,(아기 ),(사랑 ),0.010931,0.284401,0.010522,0.962617,3.384718,0.007413,19.142277
17,(별 ),(하트 ),0.015630,0.210747,0.011033,0.705882,3.349434,0.007739,2.683461
73,(별 ),"(행복 , 하트 )",0.015630,0.210747,0.011033,0.705882,3.349434,0.007739,2.683461
71,"(행복 , 별 )",(하트 ),0.015630,0.210747,0.011033,0.705882,3.349434,0.007739,2.683461
6,(구름 ),(사랑 ),0.011748,0.284401,0.010318,0.878261,3.088109,0.006977,5.878136
58,"(행복 , 구름 )",(사랑 ),0.011748,0.284401,0.010318,0.878261,3.088109,0.006977,5.878136
60,(구름 ),"(행복 , 사랑 )",0.011748,0.284401,0.010318,0.878261,3.088109,0.006977,5.878136
84,(우정 ),"(행복 , 사랑 )",0.013689,0.284401,0.010318,0.753731,2.650243,0.006425,2.905766


## 5. 연관 검색어 시각화

## 6. 트위터 API

## 7. 데이터랩 API